In [6]:
from langsmith import Client
from langsmith.evaluation import EvaluationResult, run_evaluator
from langsmith.schemas import Example, Run
from langchain.smith import RunEvalConfig


langsmith_client = Client()
dataset_name = "Autofix Eval Full 240314"

examples = 

NameError: name 'os' is not defined

In [7]:
import os
from github import Github
from github.Auth import Token
from tqdm.auto import tqdm

import sys
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '../src')))

from dotenv import load_dotenv
load_dotenv('../.env')

github = Github(auth=Token(token=os.environ.get('GITHUB_TOKEN')))
repo = github.get_repo('getsentry/sentry')

/Users/jennmueng/Documents/code/timeseries-analysis-service/.venv/lib/python3.11/site-packages/tqdm-4.66.1-py3.11.egg/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
from pydantic import field_serializer, BaseModel
from github.Commit import Commit
from typing import Any
from pydantic import ConfigDict, field_validator

from seer.automation.autofix.models import IssueDetails, EventDetails

class EvalItem(BaseModel):
    raw_data: dict[str, Any]
    commit: Commit
    issue: IssueDetails
    event: EventDetails

    model_config = ConfigDict(
        arbitrary_types_allowed=True
    )

    @field_serializer('commit')
    def serialize_commit(self, commit: Commit, _info):
        return commit.sha
    
    @field_validator('commit', mode="before")
    @classmethod
    def validate_commit(cls, commit: Commit | str):
        return commit if isinstance(commit, Commit) else repo.get_commit(commit)
    
class EvalItemWithDiff(EvalItem):
    diff: str

In [15]:
from seer.automation.autofix.models import RepoDefinition
from seer.automation.codebase.codebase_index import CodebaseIndex
import uuid
from sentence_transformers import SentenceTransformer

repo_definition = RepoDefinition(provider="github", owner="getsentry", name="seer")

embedding_model = SentenceTransformer("../models/autofix_embeddings_v0", trust_remote_code=True)
embedding_model.max_seq_length = 4096

for example in examples:
    eval_item = EvalItem.model_validate(example.inputs)
    
    CodebaseIndex.create(1, 1, repo_definition, uuid.uuid4(), embedding_model=embedding_model, sha='39155951aaed105fe9c80ab3c925689b8cbf2038')
    break

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
 46%|████▋     | 512/1103 [03:06<03:34,  2.75it/s]


KeyboardInterrupt: 